In [2]:
import numpy as np

**I use sigmoid function as activation for hidden layer:**

In [7]:
def sigmoid(x):
    x = 1 / (1 + np.exp(-x))
    return x

def sigmoid_grad(x):
    return (x) * (1-x)

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

**First I implement the classic Elman RNN also knows as simple recurrent network based on this work http://psych.colorado.edu/~kimlab/Elman1990.pdf . It doesnt unfold through time (truncated
backpropagation was used in the original Elmans work) so it's very simple. The context (hidden[t-1]) was simply regarded as
an additional input (on the left side of the image):**

<img src="rnn.png">

In [179]:
class SRN:
    def __init__(self, vocab_size, hidden_size, learn_rate):
        self.vocab_size = vocab_size
        
        self.Wx = np.random.randn(hidden_size, vocab_size) * 0.01 
        self.U = np.random.randn(hidden_size, hidden_size) * 0.01
        self.Wy = np.random.randn(vocab_size, hidden_size) * 0.01
        self.bias_hidden = np.zeros((hidden_size, 1))
        self.bias_y = np.zeros((vocab_size, 1))
        
        self.context = np.zeros((hidden_size,1))
        self.learn_rate = learn_rate
        
    def reset_context(self):
        self.context = np.zeros_like(self.context)
        
    def train(self, input, target):
        x = np.zeros((self.vocab_size, 1))
        x[char_to_num[input]] = 1
        target = char_to_num[target]
        
        #forward
        hidden = sigmoid(np.dot(self.Wx, x) + np.dot(self.U, self.context) + self.bias_hidden)
        self.context = np.copy(hidden)
        y = np.dot(self.Wy, hidden) + self.bias_y

        probs = softmax(y)            
        loss = -np.log(probs[target, 0])
        
        #backprop
        dy = np.copy(probs)
        dy[target] -= 1
        dbias_y = dy
        dWy = np.dot(dy, hidden.T)            

        dhidden = sigmoid_grad(hidden) * (np.dot(self.Wy.T, dy)) 
        dbias_hidden = dhidden

        dWx = np.dot(dhidden, x.T)
        dU = np.dot(dhidden, self.context.T)
        
        #param update 
         
        self.Wx += -self.learn_rate * dWx
        self.Wy += -self.learn_rate * dWy
        self.U += -self.learn_rate * dU
        self.bias_hidden += -self.learn_rate * dbias_hidden
        self.bias_y += - self.learn_rate * dbias_y
 
        return loss
    
    def predict(self, amount, start_el):
        predics = []
        x = np.zeros((self.vocab_size, 1))
        x[char_to_num[start_el]] = 1
        
        hidden = np.copy(self.context)

        for t in range(amount):
            hidden = sigmoid(np.dot(self.Wx, x) + np.dot(self.U, hidden) + self.bias_hidden)
            y = np.dot(self.Wy, hidden) + self.bias_y

            probs = softmax(y)
            ix = np.random.choice(range(self.vocab_size), p= probs.ravel())
            predics.append(ix)
            #predics.append(np.argmax(probs))

            x = np.zeros((self.vocab_size, 1))
            x[ix] = 1

        return ''.join(num_to_char[ix] for ix in predics)

In [176]:
text = open('shake.txt', 'r').read()
print "Text size:", len(text)
chars = list(set(text))
vocab_size = len(chars)
char_to_num = {ch : i for i, ch in enumerate(chars)}
num_to_char = {i : ch for i, ch in enumerate(chars)}

model_simple = SRN(vocab_size, 128, 0.1)

for epoch in range(3):
    print "Epoch:", epoch 
    loss = .0
    
    model.reset_context()
    for j in range(0, len(text) - 2):
        loss += model_simple.train(text[j], text[j+1])
        
        if ((j % 10000 == 0) & (j > 0)):
            print "Iteration: ", j, ", average loss: ", loss/10000
            print "Sample: ", model_simple.predict(100, text[j])
            print "  "
            loss = 0.0

Text size: 99993
Epoch: 0
Iteration:  10000 , average loss:  3.25361170453
Sample:  touwh :senmn doeHl:e  Eedn  theEtag loReeeG dor:
urnlbeo ,
dist lm'HG du.fali ee n  mMeole,.
lhth t 
  
Iteration:  20000 , average loss:  3.03968336464
Sample:   erI mihooTs er
:Ce smithewriciy fireIrpoc os S
Einwoyoe f s Ttkee  hIev
d f  t fAbweG egosdindt ula
  
Iteration:  30000 , average loss:  2.99101845237
Sample:  Am lfd  te.,UreAAre, ? e a?M:iralI arthou ferveeDn etse yle
IonR
he, thr bebie baenu 'sre Ith, waof

  
Iteration:  40000 , average loss:  2.94292884998
Sample:  ,innd wststle tf oNupsaen kLLhemethoreamunoeyceelrea
amalll ks er:
d l s, o j, anre cns Ero wt O-Ihr
  
Iteration:  50000 , average loss:  2.92818305171
Sample:  ,I onNSeeeeg,berehiha by h 
Un O sedanlou py drh:may  sYIuveye,t:Y y,irl ofil'dsdheN:nddikp,
e somie
  
Iteration:  60000 , average loss:  2.90854702316
Sample:  koUSchths tole aRA
IAn, reiprgk d g ha san:
saCPsso lsmeaninreow LotHecar oratmhQ
e  shefwy locod lo
  
It

**As we see it doesnt converge very well (I didn't try any advanced techniques for backpropagation). It learned to break words/lines some letters dependence but it is not able to build language model due to its simplicity. The original idea of Elman was to learn XOR sequences and simple letter sequences like these:**

In [181]:
text_ = np.array(["a"] * 10000)
text_[np.random.randint(0, 10000,3000)] = 'b'
text_[np.random.randint(0, 10000,3000)] = 'c'
text = ''.join(text_).replace('a', 'ae').replace('b', 'buu').replace('c','ciii')

chars = list(set(text))
vocab_size = len(chars)
char_to_num = {ch : i for i, ch in enumerate(chars)}
num_to_char = {i : ch for i, ch in enumerate(chars)}

model_simple = SRN(vocab_size, 32, 0.1)

for epoch in range(5):
    print "Epoch:", epoch 
    loss = .0
    
    model_simple.reset_context()
    for j in range(0, len(text) - 2):
        loss += model_simple.train(text[j], text[j+1])
        
        if ((j % 10000 == 0) & (j > 0)):
            print "Iteration: ", j, ", average loss: ", loss/10000
            print "Sample: ", model_simple.predict(100, text[j])
            print "  "
            loss = 0.0

 Epoch: 0
Iteration:  10000 , average loss:  0.825909765546
Sample:  iiiaeciciaebuuuaeaebuuubuciiciiibuuuaebuuaeciiciiiiciiiciiiciiiaeaebuuciiciiiiibuuuaecibuuuaeciciici
  
Iteration:  20000 , average loss:  0.563045342452
Sample:  aebuuciiiaeciaeaebuuaeaeciiiciiiiciiibuuuaeaeaeaeaeaeaeciiiaeaeaeaeaeaeaeaebuuaebuuaeaeaeaeaeaeciiii
  
Epoch: 1
Iteration:  10000 , average loss:  0.514185060425
Sample:  ibuuuaeciiibuuaebuuuaeciiiaeciiciiiaeciiciiiciiaeaebuuuaeciiibuuuaebuuciiiciiiiaebuuciiibuubuuuaecii
  
Iteration:  20000 , average loss:  0.529015320716
Sample:  ciiiiiaeaeaeaeaeaeciiiciiiciiiaebuuaeaeaeaeaeaebuuciiiuaeaeaeciiiiciiiiaebuuciiiciiiciiiaeaeaeaeciia
  
Epoch: 2
Iteration:  10000 , average loss:  0.581612858615
Sample:  iaeciaeaeaeciiiciiiiciiiciiiiciiiiaeaebuuaeaebuuaeaeciiiaeciiiaebuubuciiaeaeciiiiibuuucaeaeaeciiiici
  
Iteration:  20000 , average loss:  0.562677598285
Sample:  abuuaeciiciiiciiiiaeciibuuciaebuuciiaeciiiibuuciciiiaeaeaeaebuuaeaeaebuuaeciiibuuuaebuuaeciiiciiia

**We see it works much better for simple sequences.
Now let's go back to Shakespeare and let's try to advance our model with backpropagation through time (https://en.wikipedia.org/wiki/Backpropagation_through_time). The idea is to backpropagate errors through layers for fixed size batches. We go forward through batch and save all weights. Then we "unfold" the layers backpropagating errors "through time". The import derivative we add is dhidden_next that is the error from t+1-th step to t-th step.**

In [161]:
class RNN(SRN):   
    def __init__(self, vocab_size, hidden_size, learn_rate):
        self.vocab_size = vocab_size
        
        self.Wx = np.random.randn(hidden_size, vocab_size) * 0.01 
        self.U = np.random.randn(hidden_size, hidden_size) * 0.01
        self.Wy = np.random.randn(vocab_size, hidden_size) * 0.01
        self.bias_hidden = np.zeros((hidden_size, 1))
        self.bias_y = np.zeros((vocab_size, 1))
        
        self.context = np.zeros((hidden_size,1))
        self.learn_rate = learn_rate
        
        #we save squared weights for gradients for adagrad 
        self.adaU = np.zeros((hidden_size, hidden_size))
        self.adaWx = np.zeros((hidden_size, vocab_size))
        self.adaWy = np.zeros((vocab_size, hidden_size))
        self.adabias_hidden = np.zeros((hidden_size, 1))
        self.adabias_y = np.zeros((vocab_size, 1))
        
    def train_batch(self, inputs, targets):
        batch_loss = 0.0
        #history
        x = {}
        y = {}
        hidden = {}
        hidden[-1] = np.copy(self.context)
        probs = {}
        
        #init incremental gradients
        dWx = np.zeros_like(self.Wx)
        dU = np.zeros_like(self.U)
        dWy = np.zeros_like(self.Wy)
        dbias_hidden = np.zeros_like(self.bias_hidden)
        dbias_y = np.zeros_like(self.bias_y)
        dhidden_next = np.zeros_like(self.context)
        
        #forward
        for t in xrange(len(inputs)):
            x[t] = np.zeros((self.vocab_size, 1))
            x[t][inputs[t]] = 1
            
            hidden[t] = sigmoid(np.dot(self.Wx, x[t]) + np.dot(self.U, hidden[t-1]) + self.bias_hidden)
            y[t] = np.dot(self.Wy, hidden[t]) + self.bias_y
            
            probs[t] = softmax(y[t])
            batch_loss +=  -np.log(probs[t][targets[t], 0])
            
        #backprop   
        for t in reversed(xrange(len(inputs))):
            dy = np.copy(probs[t])
            dy[targets[t]] -= 1
            
            dbias_y += dy       
            dWy += np.dot(dy, hidden[t].T)
            
            dhidden = sigmoid_grad(hidden[t]) * ((np.dot(self.Wy.T, dy)) + dhidden_next)
            
            dbias_hidden += dhidden
            dWx += np.dot(dhidden, x[t].T)
            dU += np.dot(dhidden, hidden[t-1].T)
            
            dhidden_next = np.dot(self.U.T, dhidden)
            
        #param update 
        for dparam in [dWx, dU, dWy, dbias_hidden, dbias_y]:
            np.clip(dparam, -1, 1, out=dparam)
            
        #update RNN parameters according to Adagrad
        for param, dparam, adaparam in zip([self.U, self.Wx, self.Wy, self.bias_hidden, self.bias_y], \
                                [dU, dWx, dWy, dbias_hidden, dbias_y], \
                                [self.adaW_hh, self.adaW_xh, self.adaW_hy, self.adab_h, self.adab_y]):
            adaparam += dparam * dparam
            param += -self.learn_rate * dparam/np.sqrt(adaparam + 1e-8) 
            
        #self.Wx += -self.learn_rate * dWx
        #self.Wy += -self.learn_rate * dWy
        #self.U += -self.learn_rate * dU
        #self.bias_hidden += -self.learn_rate * dbias_hidden
        #self.bias_y += - self.learn_rate * dbias_y
        
        self.context = hidden[len(x)-1]
        return batch_loss

**There are two main problems here: vanishing and exploding grad descent problems (http://proceedings.mlr.press/v28/pascanu13.pdf). Exploding problem could be fixed clipping gradients to const value (-1 and 1 in my case). Also I didnt successed updating gradients as in the first implementation (It didn't converge at all!) so I updated that part with Adagrad (https://en.wikipedia.org/wiki/Stochastic_gradient_descent#AdaGrad) which use some kind of sparsity param update**

**The batch_size is also the amount of layers we backpropagate to**

In [185]:
text = open('shake.txt', 'r').read()
chars = list(set(text))
vocab_size = len(chars)
char_to_num = {ch : i for i, ch in enumerate(chars)}
num_to_char = {i : ch for i, ch in enumerate(chars)}

batch_size = 15
model = RNN(vocab_size, 128, 0.1)

#I train only for 5 epochs but it still converges so we can increase this one!
for epoch in range(5):
    print "Epoch:", epoch 
    loss = .0
    model.reset_context()
    
    for j in range(0, len(text)/batch_size):
        X = [char_to_num[ch] for ch in text[j * batch_size:(j+1) * batch_size]]
        Y = [char_to_num[ch] for ch in text[j * batch_size+1:(j+1) * batch_size + 1]]
        loss += model.train_batch(X, Y)
        
        if ((j % 1000 == 0) & (j > 0)):
            print "Iteration: ", j, ", batch loss: ", loss/1000
            print "Sample: ", model.predict(100, text[j])
            print "  "
            loss = 0.0

Epoch: 0
Iteration:  1000 , batch loss:  42.463785534
Sample:  aur gat, ghiw att fetse TutNNisr the derint bee fuire toeunseot euw'de
youd the kautiy ny nergWse of
  
Iteration:  2000 , batch loss:  36.1611592987
Sample:  .

GCR:CLI by'll derts al foe yoip ar you; cour on prer and poos.

Mo mafrom dere faud spue hir bede
  
Iteration:  3000 , batch loss:  34.146330537
Sample:   heganer ofei;
 othed av sore, prad sile thus rithrt, s 'ftes loun'ded, coreang bes, anve anjt a I a
  
Iteration:  4000 , batch loss:  33.9993924337
Sample:  tus, on hill mattre doss,,
Hir, Heest wiald
Shensngwha and riwe have , lomes wors lut.
 br ave bet a
  
Iteration:  5000 , batch loss:  32.9134378005
Sample:  er ind arf derse ofs whis:
Bees, in alu dears: ofr uwweng riderr ind wer wills

Brifem
Jor hacr ame,
  
Iteration:  6000 , batch loss:  32.3726121883
Sample:  f a my to shee you whas hath to he gruth
I grus workzen, wher is prast undall, math weod and, mots.

  
Epoch: 1
Iteration:  1000 , batch loss:

** We see model that backpropagates in time is able to learn syntactic rules much better than the simple one. It builds language model for Shakespeare poems and plays:**

In [184]:
print model.predict(1000, text[0])

ORE:
A distus acking':
Oud bastan.

ArFOTRUS:
Do hith moy, sprefing,
Ed: toud, could,
'veald eest we it have live howste.

Werbust Moht thy fores a dosterunt
eare I wa blames ou

Wurs, I'll spferomver'd soon a would not!
Well knemest mane in share gad; ho? a grod shalk a blows,:
Of the stort Tith me, by your spight if the winger's.
The sones.

MAYs OLAUS:
My calloes in mive palily hain the yearvels: poust as youl, ands heave the gave hart
To duered:
Lo kniey hay on extome
Whe fove thy, preaver biteer have have theess. But of re. Mognend, I'll beip netlown; and have pyoed, we saver your the will not leok
And botting; bet way seigh abe uo stear do not wids glakn dat indere, And Arin widr toep in in hh morl to moth mother my bedithing
O tidrougd dowie; with toke.

FLosth LIAs:
Nell stofes gained fith inguef:
Muve he labe peach and the grotnes my the gave houl,
Ami
the I Sagwanins fight I In quegon theoud be, a gady,
The with lance whpeed bropsuce brat I and Seembung yet the cuw
Tome
Wew, 